## MF 728: Fixed Income

### Problem set # 1

#### 1. Yield Curve Construction:
Consider the following table of USD swap rates:

| Term | Rate (%) |
|------|----------|
| 1Y   | 2.8438   |
| 2Y   | 3.060    |
| 3Y   | 3.126    |
| 4Y   | 3.144    |
| 5Y   | 3.150    |
| 7Y   | 3.169    |
| 10Y  | 3.210    |
| 30Y  | 3.237    |

Note: You may assume that these swaps pay coupons semi-annually (every 6 months). For simplicity, you may use a year fraction of 0.5 in all swap coupon payments.


##### (a) Extract the constant forward rate for the first year that enables you to match the 1Y market swap rate.

In [2]:
import numpy as np
from scipy.optimize import fsolve

In [13]:
import numpy as np
from scipy.optimize import fsolve

# 将原来的内部函数转换为独立函数
def swap_present_value(f1, sw1):
    # 计算现值PV
    PV = 0.5 * sw1 / (1 + 0.5 * f1) + 0.5 * sw1 / (1 + 0.5 * f1)**2
    # 返回PV和远期利率计算的差值
    return PV - 0.5 * f1 / (1 + 0.5 * f1) - 0.5 * f1 / (1 + 0.5 * f1)**2

# 修改后的swap_to_forward_1Y函数，使用args参数传递额外的sw1参数
def swap_to_forward_1Y(sw1):
    # fsolve现在调用swap_present_value函数，并传递sw1作为参数
    forward_rate = fsolve(swap_present_value, x0=sw1, args=(sw1,))
    return forward_rate[0]

In [55]:
# 示例调用
f1 = swap_to_forward_1Y(2.8438 / 100)
print(f1)

0.028437999999999998


##### (b) Holding this first year forward rate fixed, find the forward rate from one year to two years that enables you to match the two year swap (while also matching the one year).

In [56]:
# 独立的函数计算两年期掉期的现值差异
def swap_present_value(f2, sw2, f1):
    # 计算掉期支付的现值
    PV1 = sw2 / (1 + 0.5 * f1) + sw2 / (1 + 0.5 * f1)**2 + sw2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + sw2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2)
    # 计算远期利率支付的现值
    PV2 = f1 / (1 + 0.5 * f1) + f1 / (1 + 0.5 * f1)**2 + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2)
    # 返回现值差异
    return PV1 - PV2

# 函数用于求解第二年的远期利率
def swap_to_forward_2Y(sw2, f1):
    # 使用fsolve求解第二年的远期利率
    forward_rate_2Y = fsolve(swap_present_value, x0=sw2, args=(sw2, f1))
    return forward_rate_2Y[0]

# 使用上一步计算的第一年远期利率f1
f1 = 2.8438 / 100  # 示例中给出的第一年远期利率
# 计算第二年的远期利率
f2 = swap_to_forward_2Y(3.060 / 100, f1)
# 计算第二年的贴现因子
D1 = 1 / ((1 + 0.5 * f1)**2)  # 从前一步骤继承D1的计算
D2 = D1 / ((1 + 0.5 * f2)**2)

f2, D2


(0.03283113038626899, 0.9410092691367571)

##### (c) Continue this process and extract piecewise constant forward rates for the entire curve. Comment on the forward rates vs. the swap rates.

In [21]:
def swap_to_forward_3Y(sw3,f1,f2):
    def swap_present_value(f3):
        PV=sw3/(1+0.5*f1)+sw3/(1+0.5*f1)**2+sw3/((1+0.5*f1)**2*(1+0.5*f2))+sw3/((1+0.5*f1)**2*(1+0.5*f2)**2)+sw3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+f3*0.5))+sw3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2)
        PV2=f1/(1+0.5*f1)+f1/(1+0.5*f1)**2+f2/((1+0.5*f1)**2*(1+0.5*f2))+f2/((1+0.5*f1)**2*(1+0.5*f2)**2)+f3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+f3*0.5))+f3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2)
        return PV- PV2
    forward_rate_3Y = fsolve(swap_present_value, sw3)
    return forward_rate_3Y[0]  
f3=swap_to_forward_3Y(0.03126, f1, f2)
D3=D2/(1+0.5*f3)**2

def swap_to_forward_4Y(sw4,f1,f2,f3):
    def swap_present_value(f4):
        PV=sw4/(1+0.5*f1)+sw4/(1+0.5*f1)**2+sw4/((1+0.5*f1)**2*(1+0.5*f2))+sw4/((1+0.5*f1)**2*(1+0.5*f2)**2)+sw4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+f3*0.5))+sw4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2)+sw4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4))+sw4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2)
        PV2=f1/(1+0.5*f1)+f1/(1+0.5*f1)**2+f2/((1+0.5*f1)**2*(1+0.5*f2))+f2/((1+0.5*f1)**2*(1+0.5*f2)**2)+f3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+f3*0.5))+f3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2)+f4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4))+f4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2)
        return PV- PV2
    forward_rate_4Y = fsolve(swap_present_value, sw4)
    return forward_rate_4Y[0]  
f4=swap_to_forward_4Y(0.03144, f1, f2,f3)
D4=D3/(1+0.5*f4)**2

def swap_to_forward_5Y(sw5,f1,f2,f3,f4):
    def swap_present_value(f5):
        PV=sw5*sum((1+0.5*f1)**-i for i in range(1,3))+sw5/((1+0.5*f1)**2*(1+0.5*f2))+sw5/((1+0.5*f1)**2*(1+0.5*f2)**2)+sw5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+f3*0.5))+sw5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2)+sw5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4))+sw5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2)+sw5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2*(1+0.5*f5))+sw5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2*(1+0.5*f5)**2)
        PV2=f1/(1+0.5*f1)+f1/(1+0.5*f1)**2+f2/((1+0.5*f1)**2*(1+0.5*f2))+f2/((1+0.5*f1)**2*(1+0.5*f2)**2)+f3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+f3*0.5))+f3/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2)+f4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4))+f4/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2)+f5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2*(1+0.5*f5))+f5/((1+0.5*f1)**2*(1+0.5*f2)**2*(1+0.5*f3)**2*(1+0.5*f4)**2*(1+0.5*f5)**2)
        return PV- PV2
    forward_rate_5Y = fsolve(swap_present_value, sw5)
    return forward_rate_5Y[0]  
f5=swap_to_forward_5Y(0.0315, f1, f2,f3,f4)
D5=D4/(1+0.5*f5)**2
def swap_to_forward_7Y(sw7,f1,f2,f3,f4,f5):
    def swap_present_value(f7):
        PV=sw7*sum((1+0.5*f1)**-i for i in range(1,3))+sw7*D1*sum((1+0.5*f2)**-i for i in range(1,3))+sw7*D2*sum((1+0.5*f3)**-i for i in range(1,3))+sw7*D3*sum((1+0.5*f4)**-i for i in range(1,3))+sw7*D4*sum((1+0.5*f5)**-i for i in range(1,3))+sw7*D5*sum((1+0.5*f7)**-i for i in range(1,5))
        PV2=f1*sum((1+0.5*f1)**-i for i in range(1,3))+f2*D1*sum((1+0.5*f2)**-i for i in range(1,3))+f3*D2*sum((1+0.5*f3)**-i for i in range(1,3))+f4*D3*sum((1+0.5*f4)**-i for i in range(1,3))+f5*D4*sum((1+0.5*f5)**-i for i in range(1,3))+f7*D5*sum((1+0.5*f7)**-i for i in range(1,5))
        return PV- PV2
    forward_rate_7Y = fsolve(swap_present_value, sw7)
    return forward_rate_7Y[0]  
f7=swap_to_forward_7Y(0.03169, f1, f2,f3,f4,f5)
D7=D5/(1+f7*0.5)**4

def swap_to_forward_10Y(sw10,f1,f2,f3,f4,f5,f7):
    def swap_present_value(f10):
        PV=sw10*sum((1+0.5*f1)**-i for i in range(1,3))+sw10*D1*sum((1+0.5*f2)**-i for i in range(1,3))+sw10*D2*sum((1+0.5*f3)**-i for i in range(1,3))+sw10*D3*sum((1+0.5*f4)**-i for i in range(1,3))+sw10*D4*sum((1+0.5*f5)**-i for i in range(1,3))+sw10*D5*sum((1+0.5*f7)**-i for i in range(1,5))+sw10*D7*sum((1+0.5*f10)**-i for i in range(1,7))
        PV2=f1*sum((1+0.5*f1)**-i for i in range(1,3))+f2*D1*sum((1+0.5*f2)**-i for i in range(1,3))+f3*D2*sum((1+0.5*f3)**-i for i in range(1,3))+f4*D3*sum((1+0.5*f4)**-i for i in range(1,3))+f5*D4*sum((1+0.5*f5)**-i for i in range(1,3))+f7*D5*sum((1+0.5*f7)**-i for i in range(1,5))+f10*D7*sum((1+0.5*f10)**-i for i in range(1,7))
        return PV- PV2
    forward_rate_10Y = fsolve(swap_present_value, sw10)
    return forward_rate_10Y[0]  
f10=swap_to_forward_10Y(0.0321, f1, f2,f3,f4,f5,f7)
D10=D7/(1+f10*0.5)**6

def swap_to_forward_30Y(sw30,f1,f2,f3,f4,f5,f7,f30):
    def swap_present_value(f30):
        PV=sw30*sum((1+0.5*f1)**-i for i in range(1,3))+sw30*D1*sum((1+0.5*f2)**-i for i in range(1,3))+sw30*D2*sum((1+0.5*f3)**-i for i in range(1,3))+sw30*D3*sum((1+0.5*f4)**-i for i in range(1,3))+sw30*D4*sum((1+0.5*f5)**-i for i in range(1,3))+sw30*D5*sum((1+0.5*f7)**-i for i in range(1,5))+sw30*D7*sum((1+0.5*f10)**-i for i in range(1,7))+sw30*D10*sum((1+0.5*f30)**-i for i in range(1,41))
        PV2=f1*sum((1+0.5*f1)**-i for i in range(1,3))+f2*D1*sum((1+0.5*f2)**-i for i in range(1,3))+f3*D2*sum((1+0.5*f3)**-i for i in range(1,3))+f4*D3*sum((1+0.5*f4)**-i for i in range(1,3))+f5*D4*sum((1+0.5*f5)**-i for i in range(1,3))+f7*D5*sum((1+0.5*f7)**-i for i in range(1,5))+f10*D7*sum((1+0.5*f10)**-i for i in range(1,7))+f30*D10*sum((1+0.5*f30)**-i for i in range(1,41))
        return PV- PV2
    forward_rate_30Y = fsolve(swap_present_value, sw30)
    return forward_rate_30Y[0]  
f30=swap_to_forward_30Y(0.03237, f1, f2,f3,f4,f5,f7,f10)
D30=D10/(1+f30*0.5)**20

f_list=[f1,f2,f3,f4,f5,f7,f10,f30]
f_list

[0.028437999999999998,
 0.03283113038626899,
 0.03264530551203545,
 0.03201590223021911,
 0.031760049949048276,
 0.03222150590023792,
 0.033225985137036694,
 0.03258650990142389]

In [46]:
# 独立的函数计算三年期掉期的现值差异
def swap_present_value_3Y(f3, sw3, f1, f2):
    # 计算掉期支付的现值
    PV1 = sw3 / (1 + 0.5 * f1) + sw3 / (1 + 0.5 * f1)**2 + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2) + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)) + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)**2)
    #print(PV1)
    # 计算远期利率支付的现值
    PV2 = f1 / (1 + 0.5 * f1) + f1 / (1 + 0.5 * f1)**2 + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2) + f3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)) + f3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)**2)
    print(PV2)
    # 返回现值差异
    return PV1 - PV2

'''
# 假设f1, f2, f3是已知的远期利率，sw3是三年期掉期利率
f1, f2, f3, sw3 = 0.028438, 0.03060, 0.03126, 0.03126  # 示例值

# 初始化贴现因子列表
discount_factors = [(1 + 0.5 * f1), (1 + 0.5 * f1)**2, (1 + 0.5 * f1)**2 * (1 + 0.5 * f2), (1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2, (1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3), (1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)**2]

# 计算PV1使用列表推导
PV1 = sum(sw3 / factor for factor in discount_factors)
print(PV1)
'''
# 函数用于求解第三年的远期利率
def swap_to_forward_3Y(sw3, f1, f2):
    # 使用fsolve求解第三年的远期利率
    forward_rate_3Y = fsolve(swap_present_value_3Y, x0=sw3, args=(sw3, f1, f2))
    return forward_rate_3Y[0]

# 使用前两年计算的远期利率f1和f2
sw3 = 3.126 / 100 

# 计算第三年的远期利率
f3 = swap_to_forward_3Y(sw3, f1, f2)

# 计算第三年的贴现因子
D3 = D2 / ((1 + 0.5 * f3)**2)

f3, D3

[0.1714506]
[0.1714506]
[0.1714506]
[0.1714506]
[0.17805679]
[0.17807464]
[0.17807469]
[0.17807469]


(0.034949241677597824, 0.9089640997050304)

In [42]:
# Correctly initializing given forward rates and swap rate
f1, f2, f3, sw3 = 0.028438, 0.03060, 0.03126, 0.03126

# Correct calculation for discount factors and summing for PV1
discount_factors = [
    (1 + 0.5 * f1),
    (1 + 0.5 * f1)**2,
    (1 + 0.5 * f1)**2 * (1 + 0.5 * f2),
    (1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2,
    (1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3),
    (1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)**2
]

# Calculating PV1 using list comprehension
PV1 = sum(sw3 / factor for factor in discount_factors)

PV1
discount_factors

[1.014219,
 1.0286401799609999,
 1.0443783747144033,
 1.0603573638475337,
 1.0769307494444706,
 1.0937631770582878]

In [51]:
def generate_discount_factors_expressions(years, f_rate_symbols):
    """
    Generate expressions for discount factors over a given number of years, assuming semi-annual compounding.

    Parameters:
    - years: The number of years to generate discount factors for.
    - f_rate_symbols: The symbols (as strings) representing the forward rates for each year.

    Returns:
    - A list of strings, each representing an expression for a discount factor.
    """
    discount_factors_expressions = []
    cumulative_expression = "1"  # Start with an initial value of 1 for the cumulative product

    for i in range(years * 2):  # Multiply by 2 for semi-annual periods
        if i < len(f_rate_symbols) * 2:  # Ensure we don't exceed the list of provided rate symbols
            rate_symbol = f_rate_symbols[i // 2]  # Integer division to get the correct rate symbol
            cumulative_expression += f" * (1 + 0.5 * {rate_symbol})"
        else:
            # If we run out of rate symbols, just repeat the last one
            rate_symbol = f_rate_symbols[-1]
            cumulative_expression += f" * (1 + 0.5 * {rate_symbol})"
        
        # For each period, create a new discount factor expression
        discount_factors_expressions.append(f"({cumulative_expression})")

    return discount_factors_expressions

# Example usage
years = 3  # Generating discount factors for 3 years
f_rate_symbols = ["f1", "f2", "f3"]  # Symbols for forward rates for each year

# Generate the expressions
discount_factors_expressions = generate_discount_factors_expressions(years, f_rate_symbols)

# Printing each expression for demonstration
for i, expression in enumerate(discount_factors_expressions, start=1):
    print(f"Discount Factor {i}: {expression}")


Discount Factor 1: (1 * (1 + 0.5 * f1))
Discount Factor 2: (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1))
Discount Factor 3: (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2))
Discount Factor 4: (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2) * (1 + 0.5 * f2))
Discount Factor 5: (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2) * (1 + 0.5 * f2) * (1 + 0.5 * f3))
Discount Factor 6: (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2) * (1 + 0.5 * f2) * (1 + 0.5 * f3) * (1 + 0.5 * f3))


In [74]:
import sympy as sp

def generate_discount_factors_expressions(years, f_rate_symbols):

    discount_factors = []
    
    last_term = 1
    for i in range(years*2):
        rate_symbol = sp.symbols(f_rate_symbols[i//2]) if i//2 < len(f_rate_symbols) else sp.symbols(f_rate_symbols[-1])
        term = last_term * (1 + 0.5*rate_symbol)
        discount_factors.append(term)
        last_term = term
        
    return discount_factors

years = 3
f_rate_symbols = ["f1", "f2", "f3"]  

discount_factors = generate_discount_factors_expressions(years, f_rate_symbols)
print(discount_factors)

[0.5*f1 + 1, (0.5*f1 + 1)**2, (0.5*f1 + 1)**2*(0.5*f2 + 1), (0.5*f1 + 1)**2*(0.5*f2 + 1)**2, (0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1), (0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1)**2]


In [79]:
import sympy as sp

def generate_discount_factors_expressions(years, f_rate_symbols):

    discount_factors = []
    
    last_term = 1
    for i in range(years*2):
        rate_symbol = sp.symbols(f_rate_symbols[i//2]) if i//2 < len(f_rate_symbols) else sp.symbols(f_rate_symbols[-1])
        term = last_term * (1 + 0.5*rate_symbol)
        discount_factors.append(f"({sp.simplify(term)})") 
        last_term = term

    return discount_factors
    
years = 3
f_rate_symbols = ["f1", "f2", "f3"]  

factors = generate_discount_factors_expressions(years, f_rate_symbols)
print(factors)

['(0.5*f1 + 1)', '((0.5*f1 + 1)**2)', '((0.5*f1 + 1)**2*(0.5*f2 + 1))', '((0.5*f1 + 1)**2*(0.5*f2 + 1)**2)', '((0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1))', '((0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1)**2)']


In [80]:
def generate_dynamic_pv2_expression(years, f_rate_symbols):
    """
    Generate a dynamic expression for calculating PV2 based on a given number of years and
    symbols representing forward rates, assuming semi-annual compounding.

    Parameters:
    - years: The number of years to generate PV2 expression for.
    - f_rate_symbols: The symbols (as strings) representing the forward rates for each year.

    Returns:
    - A string representing the dynamic expression to calculate PV2.
    """
    # Generate discount factors expressions for the given years
    discount_factors_expressions = generate_discount_factors_expressions(years, f_rate_symbols)
    
    # Initialize the list to hold each term's expression for PV2
    pv2_terms_expressions = []

    # Iterate through each discount factor expression to create the PV2 terms
    for i in range(len(discount_factors_expressions)):
        rate_symbol = f_rate_symbols[min(i // 2, len(f_rate_symbols)-1)]  # Ensure index is within bounds
        term_expression = f"{rate_symbol} / {discount_factors_expressions[i]}"
        pv2_terms_expressions.append(term_expression)

    # Join all terms to form the final PV2 expression
    pv2_expression = "PV2 = " + " + ".join(pv2_terms_expressions)

    return pv2_expression

# Example usage
years = 3  # Generating PV2 expression for 3 years
f_rate_symbols = ["f1", "f2", "f3"]  # Symbols for forward rates for each year

# Generate the PV2 expression
pv2_expression = generate_dynamic_pv2_expression(years, f_rate_symbols)

print(pv2_expression)


PV2 = f1 / (0.5*f1 + 1) + f1 / ((0.5*f1 + 1)**2) + f2 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)) + f2 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)**2) + f3 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1)) + f3 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1)**2)


PV2 = f1 / (1 + 0.5 * f1) + f1 / (1 + 0.5 * f1)**2 + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2) + f3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)) + f3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)**2)

In [84]:
def generate_swap_payment_pv_expression(years, f_rate_symbols, sw_rate_symbol):
    """
    Generate a dynamic expression for calculating the present value of swap payments (PV1)
    based on a given number of years, symbols representing forward rates, and the swap rate symbol.

    Parameters:
    - years: The number of years for the swap payments.
    - f_rate_symbols: The symbols (as strings) representing the forward rates for each year.
    - sw_rate_symbol: The symbol representing the swap rate.

    Returns:
    - A string representing the dynamic expression to calculate PV1.
    """
    # Generate discount factors expressions for the given years
    discount_factors_expressions = generate_discount_factors_expressions(years, f_rate_symbols)
    
    # Initialize the list to hold each term's expression for PV1
    pv1_terms_expressions = []

    # Iterate through each discount factor expression to create the PV1 terms
    for i, df_expression in enumerate(discount_factors_expressions, start=1):
        # Create the term expression for PV1 using the swap rate symbol and the discount factor expression
        term_expression = f"{sw_rate_symbol} / {df_expression}"
        pv1_terms_expressions.append(term_expression)

    # Join all terms to form the final PV1 expression
    pv1_expression = "PV1 = " + " + ".join(pv1_terms_expressions)

    return pv1_expression

# Example usage
years = 3  # Generating PV1 expression for 3 years
f_rate_symbols = ["f1", "f2", "f3"]  # Symbols for forward rates for each year
sw_rate_symbol = "sw3"  # Symbol for the swap rate

# Generate the PV1 expression
pv1_expression = generate_swap_payment_pv_expression(years, f_rate_symbols, sw_rate_symbol)

print(pv1_expression)

PV1 = sw3 / (0.5*f1 + 1) + sw3 / ((0.5*f1 + 1)**2) + sw3 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)) + sw3 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)**2) + sw3 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1)) + sw3 / ((0.5*f1 + 1)**2*(0.5*f2 + 1)**2*(0.5*f3 + 1)**2)


AttributeError: 'str' object has no attribute 'type'

In [86]:
import sympy as sp
from scipy.optimize import fsolve

def swap_to_forward_rate(years, f_rate_values, sw_rate_value):
    """
    Solve for the last year's forward rate given swap rate, forward rates for previous years, and total years.

    Parameters:
    - years: Total number of years for the swap.
    - f_rate_values: List of forward rates for the years leading up to the last year. Last year's rate is assumed unknown.
    - sw_rate_value: Swap rate value.

    Returns:
    - The solved forward rate for the last year.
    """
    # Define symbols for all forward rates up to the given years
    f_rate_symbols = [sp.symbols(f'f{i+1}') for i in range(years)]
    
    # Substitute known forward rate values into the symbols list
    for i, rate in enumerate(f_rate_values):
        f_rate_symbols[i] = rate
    
    # Generate the last year's forward rate symbol
    last_year_f_rate_symbol = f_rate_symbols[-1]

    # 获取表达式对象
    pv1_expression = generate_swap_payment_pv_expression(years, f_rate_symbols[:-1], "sw")
    pv2_expression = generate_dynamic_pv2_expression(years, f_rate_symbols)
    
    '''
    # Generate PV1 and PV2 expressions
    pv1_expression = generate_swap_payment_pv_expression(years, f_rate_symbols[:-1], "sw")
    pv2_expression = generate_dynamic_pv2_expression(years, f_rate_symbols)
    '''
    # Substitute known values into PV1 and PV2 expressions
    pv1_str = sp.sympify(pv1_expression.replace("PV1 = ", "").replace("sw", str(sw_rate_value)))
    pv2_str = sp.sympify(pv2_expression.replace("PV2 = ", ""))

    # 用sympify转换字符串为表达式
    PV1 = sp.sympify(pv1_str)
    PV2 = sp.sympify(pv2_str)
    
    # Define the difference function
    diff_expression = PV1 - PV2
    
    # Convert sympy expression to a function usable by fsolve
    diff_func = sp.lambdify(last_year_f_rate_symbol, diff_expression, 'numpy')
    
    # Solve for the unknown forward rate
    f_rate_solution, = fsolve(diff_func, x0=0.03)  # Assuming initial guess as 3%
    
    return f_rate_solution

# Example usage
years = 3  # Total years for the swap
f1 = swap_to_forward_1Y(2.8438 / 100)
f2 = swap_to_forward_2Y(3.060 / 100, f1)
f_rate_values = [f1, f2]  # Known forward rates for the first two years
sw_rate_value = 3.126 / 100  # Swap rate

# Solve for the last year's forward rate
f3_solution = swap_to_forward_rate(years, f_rate_values, sw_rate_value)

print(f"The solved forward rate for the last year: {f3_solution}")


TypeError: 'numpy.float64' object is not iterable

PV1 = sw3 / (1 + 0.5 * f1) + sw3 / (1 + 0.5 * f1)**2 + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2) + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)) + sw3 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2 * (1 + 0.5 * f3)**2)

In [85]:
import sympy as sp
from scipy.optimize import fsolve

def swap_to_forward_rate(years, f_rate_values, sw_rate_value):
    """
    Solve for the last year's forward rate given swap rate, forward rates for previous years, and total years.

    Parameters:
    - years: Total number of years for the swap.
    - f_rate_values: List of forward rates for the years leading up to the last year. Last year's rate is assumed unknown.
    - sw_rate_value: Swap rate value.

    Returns:
    - The solved forward rate for the last year.
    """
    # Define symbols for all forward rates up to the given years
    f_rate_symbols = [sp.symbols(f'f{i+1}') for i in range(years)]
    
    # Substitute known forward rate values into the symbols list
    for i, rate in enumerate(f_rate_values):
        f_rate_symbols[i] = rate
    
    # Generate the last year's forward rate symbol
    last_year_f_rate_symbol = f_rate_symbols[-1]

    # 获取表达式对象
    pv1_expr = generate_swap_payment_pv_expression(years, f_rate_symbols[:-1], "sw")
    pv2_expr = generate_dynamic_pv2_expression(years, f_rate_symbols)
    
    # 用sympify转换字符串为表达式
    PV1 = sp.sympify(pv1_str)
    PV2 = sp.sympify(pv2_str)
    
    '''
    # Generate PV1 and PV2 expressions
    pv1_expression = generate_swap_payment_pv_expression(years, f_rate_symbols[:-1], "sw")
    pv2_expression = generate_dynamic_pv2_expression(years, f_rate_symbols)
    '''
    # Substitute known values into PV1 and PV2 expressions
    PV1 = sp.sympify(pv1_expression.replace("PV1 = ", "").replace("sw", str(sw_rate_value)))
    PV2 = sp.sympify(pv2_expression.replace("PV2 = ", ""))
    
    # Define the difference function
    diff_expression = PV1 - PV2
    
    # Convert sympy expression to a function usable by fsolve
    diff_func = sp.lambdify(last_year_f_rate_symbol, diff_expression, 'numpy')
    
    # Solve for the unknown forward rate
    f_rate_solution, = fsolve(diff_func, x0=0.03)  # Assuming initial guess as 3%
    
    return f_rate_solution

# Example usage
years = 3  # Total years for the swap
f1 = swap_to_forward_1Y(2.8438 / 100)
f2 = swap_to_forward_2Y(3.060 / 100, f1)
f_rate_values = [f1, f2]  # Known forward rates for the first two years
sw_rate_value = 3.126 / 100  # Swap rate

# Solve for the last year's forward rate
f3_solution = swap_to_forward_rate(years, f_rate_values, sw_rate_value)

print(f"The solved forward rate for the last year: {f3_solution}")


TypeError: 'numpy.float64' object is not iterable

0.03264530551203545

[0.028437999999999998,
 0.03283113038626899,
 0.03264530551203545,
 0.03201590223021911,
 0.031760049949048276,
 0.03222150590023792,
 0.033225985137036694,
 0.03258650990142389]

In [67]:
f1 = swap_to_forward_1Y(2.8438 / 100)

f2 = swap_to_forward_rate(2, [f1], swap_rates[2])
f2

0.03293280453785824

In [69]:
f2 = swap_to_forward_2Y(3.060 / 100, f1)
f2 

0.03283113038626899

In [70]:
years = 2
f_rate_symbols = ["f1", "f2"] 
sw_rate_symbol = "sw2"

pv_expression = generate_swap_payment_pv_expression(years, f_rate_symbols, sw_rate_symbol)

print(pv_expression)

PV1 = sw2 / (1 * (1 + 0.5 * f1)) + sw2 / (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1)) + sw2 / (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2)) + sw2 / (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2) * (1 + 0.5 * f2))


PV1 = sw2 / (1 + 0.5 * f1) + sw2 / (1 + 0.5 * f1)**2 + sw2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + sw2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2)


In [71]:
years = 2
f_rate_symbols = ["f1", "f2"]  

pv2_expression = generate_dynamic_pv2_expression(years, f_rate_symbols)

print(pv2_expression)

PV2 = f1 / (1 * (1 + 0.5 * f1)) + f1 / (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1)) + f2 / (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2)) + f2 / (1 * (1 + 0.5 * f1) * (1 + 0.5 * f1) * (1 + 0.5 * f2) * (1 + 0.5 * f2))


PV2 = f1 / (1 + 0.5 * f1) + f1 / (1 + 0.5 * f1)**2 + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)) + f2 / ((1 + 0.5 * f1)**2 * (1 + 0.5 * f2)**2)

In [64]:
# 定义年限的Swap利率  
swap_rates = {
    2: 3.060 / 100,  
    3: 3.126 / 100,
    4: 3.144 / 100,
    5: 3.150 / 100,
    7: 3.169 / 100,  
    10: 3.210 / 100, 
    30: 3.237 / 100   
}

# 初始远期利率f1    
f1 = 2.8438 / 100  

# 计算f2
f2 = swap_to_forward_rate(2, [f1], swap_rates[2])

# 计算f3  
f3 = swap_to_forward_rate(3, [f1, f2], swap_rates[3])

# 计算f4
f4 = swap_to_forward_rate(4, [f1, f2, f3], swap_rates[4])

# 计算f5  
f5 = swap_to_forward_rate(5, [f1, f2, f3, f4], swap_rates[5])

# 计算f7
f7 = swap_to_forward_rate(7, [f1, f2, f3, f4, f5], swap_rates[7])  

# 计算f10
f10 = swap_to_forward_rate(10, [f1, f2, f3, f4, f5, f7], swap_rates[10])  

# 计算f30
f30 = swap_to_forward_rate(30, [f1, f2, f3, f4, f5, f7, f10], swap_rates[30])

print(f1, f2, f3, f4, f5, f7, f10, f30)

TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'

error: Result from function call is not a proper array of floats.